<a href="https://colab.research.google.com/github/ykitaguchi77/FundusPhoto/blob/main/SeqNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Running Github codes (SeqNet) in Google Colab



In [1]:
#google driveをcolabolatoryにマウント
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
#Githubよりリポジトリをcloneする
!git clone https://github.com/conscienceli/SeqNet.git

Cloning into 'SeqNet'...
remote: Enumerating objects: 48, done.
remote: Counting objects: 100% (48/48), done.
remote: Compressing objects: 100% (36/36), done.
remote: Total 48 (delta 15), reused 41 (delta 10), pack-reused 0
Unpacking objects: 100% (48/48), done.


#作業ディレクトリを移動

In [4]:
import os
import shutil

path = '/content/SeqNet'

#作業ディレクトリをpathに移動する
os.chdir(path)

#作業ディレクトリ直下のファイルを確認
!ls

LICENSE  predict.py  requirements-conda.txt  train.py
pics	 README.md   requirements-pip.txt    utils


In [5]:
#評価画像用のフォルダを作成
data_dir = "./data/test_images/"

if os.path.exists(data_dir):
    shutil.rmtree(data_dir)
os.makedirs(data_dir)

#copy example data
shutil.copy("/content/drive/MyDrive/Deep_learning/FundusPhoto/cropped_img/img00085008_00_1R.jpg", "./data/test_images/image1.jpg")

'./data/test_images/image1.jpg'

In [6]:
#Download pretrained model from Gdrive
import requests

def download_file_from_google_drive(id, destination):
    URL = "https://docs.google.com/uc?export=download"

    session = requests.Session()

    response = session.get(URL, params = { 'id' : id }, stream = True)
    token = get_confirm_token(response)

    if token:
        params = { 'id' : id, 'confirm' : token }
        response = session.get(URL, params = params, stream = True)

    save_response_content(response, destination)    

def get_confirm_token(response):
    for key, value in response.cookies.items():
        if key.startswith('download_warning'):
            return value

    return None

def save_response_content(response, destination):
    CHUNK_SIZE = 32768

    with open(destination, "wb") as f:
        for chunk in response.iter_content(CHUNK_SIZE):
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)

file_id = '1OYjzu0gixtga6e7Rvb2mZoSSYJkXWRNB'

os.makedirs('./trained_model/ALL', exist_ok=True)
destination = './trained_model/ALL/Final_Emer_Iteration_3_cropsize_128_epochs_200.hdf5'
download_file_from_google_drive(file_id, destination)

In [ ]:
!pip3 install --upgrade tensorflow==1.15.0
!pip3 install --upgrade keras==2.3.1
!pip3 install --upgrade h5py==2.10.0


In [ ]:
#stride_sizeを増やす事によるタイムコスト削減
!cat -n predict.py # -n はnumberの意味
!sed -i -e "133c predict(batch_size=24, epochs=200, iteration=3, stride_size=10, crop_size=128," predict.py

In [ ]:
"""
#ここは不要
!cat -n requirements-pip.txt # -n はnumberの意味

!sed -i -e "43c protobuf==3.11.3" requirements-pip.txt # protobufはバージョン違いしかないので変更
!sed -i -e "47d" requirements-pip.txt # pyGPUはサポートされていないので除外

!cat -n requirements-pip.txt # -n はnumberの意味

!pip3 install -r requirements-pip.txt
"""


#**Run codes**

In [ ]:
!python predict.py -i ./data/test_images/ -o ./output/